In [9]:
import numpy as np
import skimage as sk
import skimage.io as skio
import cv2 
from scipy import signal
from scipy.spatial import Delaunay
from scipy import interpolate
import scipy
from skimage import color
import math
import matplotlib
from matplotlib import pyplot as plt
%matplotlib qt
from skimage.draw import polygon

import skimage.transform as sktr


import sys, re
from os import listdir
from os.path import isfile, isdir, join
import os

## Defining Correspondencies 

In [62]:
def get_points(im1, im2):
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.close()
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C, _ = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),
             (0, 0)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1, b1 = im1.shape
    h2, w2, b2 = im2.shape
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    if dscale < 1:
        im1 = sktr.rescale(im1, dscale)
    else:
        im2 = sktr.rescale(im2, 1./dscale)
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1, c1 = im1.shape
    h2, w2, c2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :, :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :, :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.)), :]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.)), :]
    assert im1.shape == im2.shape
    return im1, im2


def align_images(im1, im2):
    pts = get_points(im1, im2)
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2



In [63]:
def correspondencies(im, num):
    plt.imshow(im)
    points = plt.ginput(num, timeout=200)
    return points

In [ ]:
name = 'lucy.jpg'
im1 = skio.imread(name) 
im1 = sk.img_as_float(im1)

name = 'yanchu.jpg'
im2 = skio.imread(name) 
im2 = sk.img_as_float(im2)

In [ ]:
im1_aligned, im2_aligned = align_images(im1, im2)
im1 = im1_aligned[80:,:450]
im2 = im2_aligned[80:,:450]

In [46]:
points1 = correspondencies(im1, 44)

In [47]:
points2 = correspondencies(im2, 44)

In [180]:
file = open("lucy_points.txt", "w")
file.write(str(points1))
file.close()

file = open("yanchu_points.txt", "w")
file.write(str(points2))
file.close()

skio.imsave("lucy_aligned.jpg", im1)
skio.imsave("yanchu_aligned.jpg", im2)

C:\Users\wangkehua\Anaconda3\envs\booknlp\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [207]:
# name = 'lucy_aligned.jpg'
# im1 = skio.imread(name) 
# im1 = sk.img_as_float(im1)

In [51]:
points1 = np.array(points1)
points2 = np.array(points2)

tri1 = Delaunay(points1)
tri2 = Delaunay(points2)

plt.triplot(points1[:,0], points1[:,1], tri1.simplices)
plt.plot(points1[:,0], points1[:,1], 'o')
plt.imshow(im1)
plt.axis('off')
plt.savefig("lucy_tri.jpg", dpi = 200)
plt.show()

In [50]:
plt.triplot(points2[:,0], points2[:,1], tri2.simplices)
plt.plot(points2[:,0], points2[:,1], 'o')
plt.imshow(im2)
plt.axis('off')
plt.savefig("yanchu_tri.jpg", dpi = 200)
plt.show()

## Compute the Mid-way Face

In [64]:
def midway(points1, points2, alpha):
    mid_points = (1 - alpha) * points1 + alpha * points2
    tri_mid = Delaunay(mid_points)
    return np.array(mid_points), tri_mid

In [65]:
def find_middle(alpha, im1, im2, points1, points2):
    mid, tri_mid = midway(points1, points2, alpha)
    mid_face_all = []
    for channel in range(3):
        mid_face = np.zeros([im1.shape[0], im1.shape[1]])
        for i in range(len(tri_mid.simplices)):
            triangle1 = np.array([list(points1[k]) for k in tri_mid.simplices[i]])
            triangle2 = np.array([list(points2[k]) for k in tri_mid.simplices[i]])
            mid_triangle = mid[tri_mid.simplices[i]]

            A = np.vstack([triangle1.T, np.array([1, 1, 1])])
            B = np.vstack([triangle2.T, np.array([1, 1, 1])])
            T1 = np.vstack([mid_triangle.T @ np.linalg.inv(A), np.array([0, 0, 1])])
            T2 = np.vstack([mid_triangle.T @ np.linalg.inv(B), np.array([0, 0, 1])])

            row, col = polygon(mid_triangle[:,0], mid_triangle[:,1])
            M = np.vstack([row, col, np.ones(len(row))])
            im1_interp = np.linalg.inv(T1) @ M
            im2_interp = np.linalg.inv(T2) @ M
            
            #Interpolate
            f1 = interpolate.interp2d(range(im1.shape[1]), range(im1.shape[0]), im1[:,:,channel])
            f2 = interpolate.interp2d(range(im2.shape[1]), range(im2.shape[0]), im2[:,:,channel])
            for j in range(len(row)):
                mid_face[col[j], row[j]] += (1 - alpha) * f1(im1_interp[0, j], im1_interp[1, j])
                mid_face[col[j], row[j]] += alpha * f2(im2_interp[0, j], im2_interp[1, j])


                
        mid_face_all.append(np.clip(mid_face, 0, 1))

           
    return np.dstack([mid_face_all[0], mid_face_all[1], mid_face_all[2]])
        
   

In [81]:
mid_face = find_middle(0.5, im1, im2, points1, points2)
plt.imshow(mid_face)
plt.axis('off')
plt.savefig("midface.jpg", dpi=200)

## The Morph Sequence 

In [89]:
fraction = np.round(np.linspace(0, 1, 30), 2)
for frac in fraction:
    mid_face = find_middle(frac, im1, im2, points1, points2)
    plt.imshow(mid_face)
    plt.axis('off')
    plt.savefig("img_warp/" + str(np.where(fraction == frac)[0][0]) + ".jpg", dpi=200)

In [85]:
plt.imshow(mid_face)

## The Mean Face of a Population

#### 1. Compute the mean face of whole population

In [66]:
folder = "face_data/data"
imgs = [skio.imread(join(folder, f))/255 for f in listdir(folder) if ".bmp" in join(folder, f)]

In [67]:
def extract_points():
    all_points = []
    for i in range(len(asf)):
        file = open(asf[i], "r")
        points = []
        for line in file.readlines():
            if "\t" in line:
                point = []
                point.append(float(line.split("\t")[2]) * imgs[i].shape[1])
                point.append(float(line.split("\t")[3]) * imgs[i].shape[0])
                points.append(point)
        points.append([0, imgs[i].shape[0]])
        points.append([imgs[i].shape[1], 0])
        points.append([imgs[i].shape[1], imgs[i].shape[0]])
        points.append([0, 0])
        all_points.append(np.array(points))
    return all_points

In [114]:
asf = [join(folder, f) for f in listdir(folder) if ".asf" in join(folder, f)]
all_points = extract_points()

In [115]:
def mean_shape():
    mean_points = sum(all_points) / len(all_points)
    tri_mean = Delaunay(mean_points)
    return np.array(mean_points), tri_mean

In [116]:
mid, tri_mid = mean_shape()
# plt.triplot(mid[:,0], mid[:,1], tri_mid.simplices)
# plt.plot(mid[:,0], mid[:,1], 'o')
# plt.imshow(imgs[0])

In [84]:
tri1 = Delaunay(all_points[0])

plt.triplot(all_points[0][:,0], all_points[0][:,1], tri1.simplices)
plt.plot(all_points[0][10:12,0], all_points[0][10:12,1], 'o')
plt.imshow(imgs[0])

In [71]:
def find_mean():
    mid, tri_mid = mean_shape()
    mid_face_all = []
    for channel in range(3):
        mid_face = np.zeros([imgs[0].shape[0], imgs[1].shape[1]])
        for i in range(len(tri_mid.simplices)):
            
            mid_triangle = mid[tri_mid.simplices[i]]
            row, col = polygon(mid_triangle[:,0], mid_triangle[:,1])
            M = np.vstack([row, col, np.ones(len(row))])
            
            for j in range(len(imgs)):
                triangle = np.array([list(all_points[j][k]) for k in tri_mid.simplices[i]])
                A = np.vstack([triangle.T, np.array([1, 1, 1])])
                T = np.vstack([mid_triangle.T @ np.linalg.inv(A), np.array([0, 0, 1])])
                im_interp = np.linalg.inv(T) @ M
                
                
                #Interpolate
                f = interpolate.interp2d(range(imgs[j].shape[1]), range(imgs[j].shape[0]), imgs[j][:,:,channel])
                for k in range(len(row)):
                    mid_face[col[k], row[k]] += f(im_interp[0, k], im_interp[1, k]) / len(imgs)

        mid_face_all.append(mid_face)

           
    return np.dstack([mid_face_all[0], mid_face_all[1], mid_face_all[2]])
        
    

In [171]:
mean_face = find_mean()
plt.imshow(mean_face)
plt.axis('off')
plt.savefig("mean_population_face.jpg", dpi=200)

#### 2. Morph each face into mean face

In [164]:
def morph_into_mean(imgs,points,  all_imgs, name=None):
    mid, tri_mid = mean_shape()

    for j in range(len(imgs)):
        mid_face_all = []
        for channel in range(3):
            mid_face = np.zeros([all_imgs[0].shape[0], all_imgs[0].shape[1]])
            for i in range(len(tri_mid.simplices)):

                mid_triangle = mid[tri_mid.simplices[i]]
                row, col = polygon(mid_triangle[:,0], mid_triangle[:,1])
                M = np.vstack([row, col, np.ones(len(row))])

                triangle = np.array([list(points[j][k]) for k in tri_mid.simplices[i]])
                A = np.vstack([triangle.T, np.array([1, 1, 1])])
                T = np.vstack([mid_triangle.T @ np.linalg.inv(A), np.array([0, 0, 1])])
                im_interp = np.linalg.inv(T) @ M

                #Interpolate
                f = interpolate.interp2d(range(imgs[j].shape[1]), range(imgs[j].shape[0]), imgs[j][:,:,channel])
                for k in range(len(row)):
                    mid_face[col[k], row[k]] += f(im_interp[0, k], im_interp[1, k])

            mid_face_all.append(mid_face)
            
        warp_img = np.dstack([mid_face_all[0], mid_face_all[1], mid_face_all[2]])
        plt.imshow(warp_img)
        plt.axis('off')
        if len(imgs) == 1:
            plt.savefig("face_data/" + name, dpi=200)
        plt.savefig("face_data/" + str(j) + ".jpg", dpi=200)
        
        
 

In [133]:
# morph_into_mean(imgs, all_points, imgs)   

#### Morph myself into mean face

In [88]:
points_me = correspondencies(im1, 58)
points_me = np.array(points_me)

In [160]:
corners = np.array([[0, im1.shape[0]],
                    [im1.shape[1], 0], 
                    [im1.shape[1], im1.shape[0]],  
                    [0, 0]])

points_me = np.vstack([points_me, corners])

In [165]:
morph_into_mean([im1], [points_me], imgs, "my_into_mean.jpg")

#### Morph mean face into my geometry 

In [265]:
name = 'mean_population_face_aligned.jpg'
im_mean = skio.imread(name) 
im_mean = sk.img_as_float(im_mean)


In [266]:
im_mean, im1 = align_images(im_mean, im1)

Please select 2 points in each image for alignment.


C:\Users\wangkehua\Anaconda3\envs\booknlp\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [267]:
im1 = im1[15:,:310]
im_mean = im_mean[15:,:310]

In [268]:
points_mean = correspondencies(im_mean, 58)
points_me = np.array(points_me)

In [269]:

corners = np.array([[0, im_mean.shape[0]],
                    [im_mean.shape[1], 0], 
                    [im_mean.shape[1],im_mean.shape[0]],  
                    [0, 0]])

points_mean = np.vstack([points_mean, corners])

In [281]:
points_me = correspondencies(im1, 58)
points_me = np.array(points_me)

In [282]:
corners = np.array([[0, im1.shape[0]],
                    [im1.shape[1], 0], 
                    [im1.shape[1], im1.shape[0]],  
                    [0, 0]])

points_me = np.vstack([points_me, corners])

In [306]:
def morph_mean_into_me(im_mean, points_mean, im_me, points_me, name):
    tri_mid = Delaunay(points_me)

    mid_face_all = []
    for channel in range(3):
        mid_face = np.zeros([im_mean.shape[0], im_me.shape[1]])
        for i in range(len(tri_mid.simplices)):

            mid_triangle = points_me[tri_mid.simplices[i]]
#             print(mid_triangle)
            row, col = polygon(mid_triangle[:,0], mid_triangle[:,1])
            M = np.vstack([row, col, np.ones(len(row))])

            triangle = np.array([list(points_mean[k]) for k in tri_mid.simplices[i]])
            A = np.vstack([triangle.T, np.array([1, 1, 1])])
            T = np.vstack([mid_triangle.T @ np.linalg.inv(A), np.array([0, 0, 1])])
            im_interp = np.linalg.inv(T) @ M
#             print(row, col)
            #Interpolate
            f = interpolate.interp2d(range(im_mean.shape[1]), range(im_mean.shape[0]), im_mean[:,:,channel])
#             print(mid_face.shape)
            for k in range(len(row)):
#                 print(col[k], row[k])
                mid_face[col[k], row[k]] += f(im_interp[0, k], im_interp[1, k])

        mid_face_all.append(mid_face)

    warp_img = np.dstack([mid_face_all[0], mid_face_all[1], mid_face_all[2]])
    plt.imshow(warp_img)
    plt.axis('off')
    plt.savefig(name, dpi=200)

In [307]:
morph_mean_into_me(im_mean, points_mean, im1, points_me, "face_data/mean_into_me.jpg")

## Caricatures 

In [316]:
alpha = 2
dx = points_me - points_mean
points_new_me = points_mean + alpha * dx
morph_mean_into_me(im1, points_me, im1, points_new_me, name="face_data/caricature1.jpg")

In [317]:
alpha = -1
dx = points_me - points_mean
points_new_me = points_mean + alpha * dx
morph_mean_into_me(im1, points_me, im1, points_new_me, name="face_data/caricature2.jpg")

## Bells & Whistles

In [156]:
name = 'ken.jpg'
im_ken = skio.imread(name) 
im_ken = sk.img_as_float(im_ken)

name = 'lucy2.jpg'
im_lucy = skio.imread(name) 
im_lucy = sk.img_as_float(im_lucy)


In [195]:
im_ken_aligned, im_lucy_aligned = align_images(im_ken, im_lucy)

Please select 2 points in each image for alignment.


C:\Users\wangkehua\Anaconda3\envs\booknlp\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [206]:
im_ken = im_ken_aligned[240:990, 110:712]
im_lucy = im_lucy_aligned[240:990, 110:712]
skio.imsave("lucy2_aligned.jpg", im_lucy)
skio.imsave("ken_aligned.jpg", im_ken)

C:\Users\wangkehua\Anaconda3\envs\booknlp\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [210]:
points_lucy = correspondencies(im_lucy, 44)

In [211]:
points_ken = correspondencies(im_ken, 44)

In [215]:
points_lucy = np.array(points_lucy)
points_ken = np.array(points_ken)
file = open("lucy2_points.txt", "w")
file.write(str(points_lucy))
file.close()

file = open("ken_points.txt", "w")
file.write(str(points_ken))
file.close()

In [76]:
im_lucy = skio.imread('lucy2_aligned.jpg')
im_ken = skio.imread('ken_aligned.jpg')
im_lucy = sk.img_as_float(im_lucy)
im_ken = sk.img_as_float(im_ken)

In [26]:
points_lucy = np.array(points_lucy)
points_ken = np.array(points_ken)

In [82]:
fraction = np.round(np.linspace(0, 1, 30), 2)[11:]
i = 11
for frac in fraction:
    mid_face = find_middle(frac, im_lucy, im_ken, points_lucy, points_ken)
    plt.imshow(mid_face)
    plt.axis('off')
    plt.savefig("1/" + str(i) + ".jpg", dpi=200)
    i += 1